In [ ]:
import os

import torch
from hydra.utils import instantiate
from monai.apps import download_url
from neuro_utils.visualize import plot_scans
from torch import nn

from diffusion_3d.chestct.autoencoder.vae.maisi.config import get_config
from diffusion_3d.datasets.ct_rate import CTRATEDataModule

In [ ]:
trained_autoencoder_path = r"/raid3/arjun/checkpoints/maisi/autoencoder_epoch273.pt"
trained_autoencoder_path_url = "https://developer.download.nvidia.com/assets/Clara/monai/tutorials/model_zoo/model_maisi_autoencoder_epoch273_alternative.pt"
if not os.path.exists(trained_autoencoder_path):
    download_url(url=trained_autoencoder_path_url, filepath=trained_autoencoder_path)

In [ ]:
state_dict = torch.load(trained_autoencoder_path, map_location='cpu')

/tmp/ipykernel_1448364/2833122154.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(trained_autoencoder_path, map_location='cpu')


In [ ]:
model_config = {
    "_target_": "monai.apps.generation.maisi.networks.autoencoderkl_maisi.AutoencoderKlMaisi",
    "spatial_dims": 3,
    "in_channels": 1,
    "out_channels": 1,
    "latent_channels": 4,
    "num_channels": [64, 128, 256],
    "num_res_blocks": [2, 2, 2],
    "norm_num_groups": 32,
    "norm_eps": 1e-06,
    "attention_levels": [False, False, False],
    "with_encoder_nonlocal_attn": False,
    "with_decoder_nonlocal_attn": False,
    "use_checkpointing": False,
    "use_convtranspose": False,
    "norm_float16": False,
    "num_splits": 1,
    "dim_split": 0,
}

In [ ]:
device = torch.device('cuda:0')
# Don't use cpu, server crashes because of mem and compute requirements

In [ ]:
model: nn.Module = instantiate(model_config)
model.load_state_dict(state_dict)
model.to(device)
model.eval()


AutoencoderKlMaisi(
  (encoder): MaisiEncoder(
    (blocks): ModuleList(
      (0): MaisiConvolution(
        (conv): Convolution(
          (conv): Conv3d(1, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        )
      )
      (1-2): 2 x MaisiResBlock(
        (norm1): MaisiGroupNorm3D(32, 64, eps=1e-06, affine=True)
        (conv1): MaisiConvolution(
          (conv): Convolution(
            (conv): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          )
        )
        (norm2): MaisiGroupNorm3D(32, 64, eps=1e-06, affine=True)
        (conv2): MaisiConvolution(
          (conv): Convolution(
            (conv): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          )
        )
        (nin_shortcut): Identity()
      )
      (3): MaisiDownsample(
        (conv): MaisiConvolution(
          (conv): Convolution(
            (conv): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2))
          )
    

In [ ]:
data_config = get_config((32, 256, 256)).data

datamodule = CTRATEDataModule(data_config)
dataloader = datamodule.val_dataloader()
len(dataloader)

valid:   0%|          | 0/500 [00:00<?, ?it/s]

No. of valid datapoints: 500


500

In [18]:
with torch.no_grad():
    for i, batch in enumerate(dataloader):
        x = batch['image']
        x = x[:5]
        print(x.shape, x.min(), x.max())
        y = model(x.to(device))[0].cpu()
        print(y.shape, y.min(), y.max())

        for i in range(x.shape[0]):
            plot_scans([x[i, 0], y[i, 0]], ["Original", "Reconstructed"])
        
        break

torch.Size([5, 1, 32, 256, 256]) metatensor(0.) metatensor(1.)
torch.Size([5, 1, 32, 256, 256]) tensor(-0.0086) tensor(1.1168)


interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…